In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import os

os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices'

In [ ]:
# Reads CSVs
def read_csv(file, encoding):
    df = pd.read_csv(file, encoding=encoding)
    df.fillna(value="", inplace=True)
    return df

# Read covid BERT documents
faq_covidbert = read_csv("data/faqs/faq_covidbert.csv", "utf8")



In [ ]:
# Documents retriever
# Retrieve documents and store them into database
from haystack.database.elasticsearch import ElasticsearchDocumentStore
from haystack.retrieve.elasticsearch import ElasticsearchRetriever

retriever = ElasticsearchRetriever()
document_store = ElasticsearchRetriever()

In [7]:
# Load GloVe embeddings
def load_glove():
    f = open('glove/glove.6B.100d.txt',encoding="utf8")
    embeddings_index = dict()
    for line in f:
        values = line.split()
        word = values[0]
        embeddings = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = embeddings
    f.close()
    print('Found {} word vectors.'.format(len(embeddings_index)))

load_glove()

Found 400000 word vectors.


In [11]:
INPUT_DIMENSION = 100 # using GloVe Embedding 100 dimensions.
OUTPUT_DIMENSION = 50 # output embedding layer 50 dimensions.
KERNEL_SIZE = 5 # Kernel size.

model = keras.Sequential()
model.add(layers.Embedding(input_dim=INPUT_DIMENSION, output_dim=OUTPUT_DIMENSION))
model.add(layers.Convolution1D(filters=INPUT_DIMENSION, kernel_size=KERNEL_SIZE))
model.add(layers.MaxPooling1D(pool_size=2, strides=None, padding="valid"))
model.add(layers.Dense(128))
model.add(layers.Convolution1D(filters=INPUT_DIMENSION, kernel_size=KERNEL_SIZE))
model.add(layers.MaxPooling1D(pool_size=2, strides=None, padding="valid"))
model.add(layers.Dense(128))
model.add(layers.LSTM(128))
model.add(layers.Dense(10))
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, None, 50)          5000      
_________________________________________________________________
conv1d_3 (Conv1D)            (None, None, 100)         25100     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, None, 100)         0         
_________________________________________________________________
dense_3 (Dense)              (None, None, 128)         12928     
_________________________________________________________________
conv1d_4 (Conv1D)            (None, None, 100)         64100     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, None, 100)         0         
_________________________________________________________________
dense_4 (Dense)              (None, None, 128)        